In [1]:
%pip install torch
%pip install transformers[torch]
%pip install ipywidgets
%pip install datasets
%pip install -U scikit-learn
%pip install -U ipywidgets
%pip install 'accelerate>=1.1.0'
%pip install numpy requests nlpaug
%pip install nltk>=3.4.5

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
!nvidia-smi

Fri Feb 27 08:39:04 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.126.09             Driver Version: 580.126.09     CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000001:00:00.0 Off |                  Off |
| N/A   31C    P8             11W /   70W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import torch
import torch.nn as nn
import pandas as pd
from transformers import Trainer, TrainingArguments
from datasets import Dataset
from itertools import product
from collections import defaultdict
from torch.nn import functional as F


pd.set_option('display.width',1000)


In [4]:
torch.cuda.empty_cache()

In [5]:
RANDOM_SEED = 1
torch.manual_seed(RANDOM_SEED)

EPOCHS = 3
MAX_LENGTH = 128
K = 5


device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [6]:
from load_data import DontPatronizeMe

DATA_DIR = 'data/'
TEST_PATH = f'{DATA_DIR}task4_test.tsv'

dpm = DontPatronizeMe(DATA_DIR, TEST_PATH)
dpm.load_task1()

data = dpm.train_task1_df

data['text'] = data['keyword'] + ' ' + data['text']

trids = pd.read_csv(f'{DATA_DIR}train_semeval_parids-labels.csv')
devids = pd.read_csv(f'{DATA_DIR}dev_semeval_parids-labels.csv')

trids['par_id'] = trids.par_id.astype(str)
devids['par_id'] = devids.par_id.astype(str)

cols = ['par_id', 'text', 'label_y', 'keyword']

trdf = trids.merge(data, on='par_id', how='left')[cols]
devdf = devids.merge(data, on='par_id', how='left')[cols]

# rename label_y to label for Trainer
trdf.rename(columns={"label_y": "labels"}, inplace=True)
devdf.rename(columns={"label_y": "labels"}, inplace=True)

# For ablation studies
og_trdf = trdf.copy()
og_devdf = devdf.copy()

# Prepend keyword to text
trdf['text'] = trdf['keyword'] + ' ' + trdf['text']
trdf.drop(columns=['keyword'], inplace=True)
devdf['text'] = devdf['keyword'] + ' ' + devdf['text']
devdf.drop(columns=['keyword'], inplace=True)

print(trdf.head())
print()

print(devdf.head())
print()

trds = Dataset.from_pandas(trdf)
print(trds)


  par_id                                               text  labels
0   4341  poor-families poor-families The scheme saw an ...       1
1   4136  homeless homeless Durban 's homeless communiti...       1
2  10352  poor-families poor-families The next immediate...       1
3   8279  vulnerable vulnerable Far more important than ...       1
4   1164  poor-families poor-families To strengthen chil...       1

  par_id                                               text  labels
0   4046  hopeless hopeless We also know that they can b...       1
1   1279  refugee refugee Pope Francis washed and kissed...       1
2   8330  refugee refugee Many refugees do n't want to b...       1
3   4063  in-need in-need "Budding chefs , like "" Fred ...       1
4   4089  homeless homeless "In a 90-degree view of his ...       1

Dataset({
    features: ['par_id', 'text', 'labels'],
    num_rows: 8375
})


In [7]:
from sklearn.model_selection import StratifiedKFold, train_test_split

VAL_SIZE = 0.1

trdf, valdf = train_test_split(trdf, test_size=VAL_SIZE, random_state=RANDOM_SEED, stratify=trdf['labels'])


In [8]:
import nlpaug.augmenter.word as naw
import nltk

nltk.download("wordnet")
nltk.download("omw-1.4")
nltk.download('averaged_perceptron_tagger_eng')

AUG_FACTOR = 2
POS_LABEL = 1

original_counts = trdf["labels"].value_counts().sort_index()
print("Before augmentation:")
print(original_counts)
print()

def augment(df):
    pos_df = df[df["labels"] == POS_LABEL].copy()

    aug = naw.SynonymAug(aug_src="wordnet")
    augmented_rows = []

    for _, row in pos_df.iterrows():
        for _ in range(AUG_FACTOR - 1):
            new_row = row.copy()
            new_row["text"] = aug.augment(row["text"])[0]
            augmented_rows.append(new_row)
        
    aug_df = pd.DataFrame(augmented_rows)
    df = pd.concat([df, aug_df], ignore_index=True)
    df = df.sample(frac=1, random_state=RANDOM_SEED).reset_index(drop=True)

    return df

trdf = augment(trdf)

counts = trdf["labels"].value_counts().sort_index()
print("After augmentation:")
print(counts)
print()


[nltk_data] Downloading package wordnet to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


Before augmentation:
labels
0    6822
1     715
Name: count, dtype: int64

After augmentation:
labels
0    6822
1    1430
Name: count, dtype: int64



In [9]:
weights = torch.tensor([counts[0]/counts[0], counts[0]/counts[1]], dtype=torch.float).to(device)
print("Class weights:", weights)
print()

Class weights: tensor([1.0000, 4.7706], device='cuda:0')



In [10]:
skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=RANDOM_SEED)

X = trdf["text"].values
y = trdf["labels"].values

folds = list(skf.split(X, y))

In [11]:
from sklearn.metrics import f1_score

class BalancedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels").view(-1)

        outputs = model(**inputs)
        logits = outputs.logits.view(-1, 2)

        loss_fn = nn.CrossEntropyLoss(weight=weights)
        loss = loss_fn(logits, labels)

        return (loss, outputs) if return_outputs else loss

def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    labels = p.label_ids
    f1 = f1_score(labels, preds, pos_label=1)
    return { "f1": f1 }



In [12]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

rob_checkpoint = "FacebookAI/roberta-base"
deb_checkpoint = "microsoft/deberta-base"

rob_tokenizer = AutoTokenizer.from_pretrained(rob_checkpoint)
deb_tokenizer = AutoTokenizer.from_pretrained(deb_checkpoint)

def tokenize(tokenizer, batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=MAX_LENGTH)

architectures = [("Roberta", rob_checkpoint, rob_tokenizer), ("Deberta", deb_checkpoint, deb_tokenizer)]


In [18]:
LRS = [2e-5, 3e-5]
BSZS = [16, 32]
WEIGHT_DECAY = 0.05
DROPOUT = 0.2

num_epochs = len(folds) * len(architectures) * len(LRS) * len(BSZS) * EPOCHS
print(f"Total training epochs: {num_epochs}")
print()

# Determine best hyperparameters using k-fold cross-validation
def hyperparameter_search():
    hp_metrics = {}
    hp_metrics["Roberta"] = defaultdict(float)
    hp_metrics["Deberta"] = defaultdict(float)

    res = {}

    for name, checkpoint, tokenizer in architectures:
        print(f"Training {name}...")
        print()

        for fold, (train_idx, val_idx) in enumerate(folds):
            print(f"Fold {fold + 1}/{K}")

            trds = Dataset.from_pandas(trdf.iloc[train_idx])
            valds = Dataset.from_pandas(trdf.iloc[val_idx])

            trds = trds.map(lambda batch: tokenize(tokenizer, batch), batched=True)
            valds = valds.map(lambda batch: tokenize(tokenizer, batch), batched=True)

            trds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
            valds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

            for lr, bsz in product(LRS, BSZS):
                model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2,
                                                                           hidden_dropout_prob=DROPOUT,
                                                                           attention_probs_dropout_prob=DROPOUT).to(device)
                training_args = TrainingArguments(
                    learning_rate=lr,
                    per_device_train_batch_size=bsz,
                    per_device_eval_batch_size=bsz,
                    num_train_epochs=EPOCHS,
                    logging_strategy="epoch",
                    eval_strategy="epoch",
                    save_strategy="no",
                    remove_unused_columns=False,
                    weight_decay=WEIGHT_DECAY,
                )

                trainer = BalancedTrainer(
                    model=model,
                    args=training_args,
                    train_dataset=trds,
                    eval_dataset=valds,
                    compute_metrics=compute_metrics
                )

                trainer.train()
                metrics = trainer.evaluate()

                model = model.cpu()

                hp_metrics[name][(lr, bsz)] += metrics["eval_f1"]
            
        best_hp = max(hp_metrics[name], key=hp_metrics[name].get)
        avg_f1 = hp_metrics[name][best_hp] / K
        print(f"Best hyperparameters for {name}: {best_hp}, Average F1: {avg_f1:.4f}")
        res[name] = {
            "best_hps": best_hp,
            "avg_f1": avg_f1
        }

    return res

Total training epochs: 120



In [11]:
tuning_res = hyperparameter_search()
print(tuning_res)

Training Roberta...

Fold 1/5


Map:   0%|          | 0/6029 [00:00<?, ? examples/s]

Map:   0%|          | 0/1508 [00:00<?, ? examples/s]

Loading weights:   0%|          | 0/197 [00:00<?, ?it/s]

RobertaForSequenceClassification LOAD REPORT from: FacebookAI/roberta-base
Key                             | Status     | 
--------------------------------+------------+-
lm_head.dense.weight            | UNEXPECTED | 
lm_head.layer_norm.weight       | UNEXPECTED | 
roberta.embeddings.position_ids | UNEXPECTED | 
lm_head.layer_norm.bias         | UNEXPECTED | 
lm_head.dense.bias              | UNEXPECTED | 
lm_head.bias                    | UNEXPECTED | 
classifier.out_proj.bias        | MISSING    | 
classifier.dense.bias           | MISSING    | 
classifier.dense.weight         | MISSING    | 
classifier.out_proj.weight      | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


...

Epoch,Training Loss,Validation Loss,F1
1,0.668826,0.700549,0.410256
2,0.593890,0.675493,0.412556
3,0.475445,0.636739,0.487952


Loading weights:   0%|          | 0/196 [00:00<?, ?it/s]

DebertaForSequenceClassification LOAD REPORT from: microsoft/deberta-base
Key                                     | Status     | 
----------------------------------------+------------+-
lm_predictions.lm_head.dense.bias       | UNEXPECTED | 
lm_predictions.lm_head.LayerNorm.bias   | UNEXPECTED | 
lm_predictions.lm_head.LayerNorm.weight | UNEXPECTED | 
lm_predictions.lm_head.dense.weight     | UNEXPECTED | 
lm_predictions.lm_head.bias             | UNEXPECTED | 
classifier.weight                       | MISSING    | 
pooler.dense.bias                       | MISSING    | 
classifier.bias                         | MISSING    | 
pooler.dense.weight                     | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


Epoch,Training Loss,Validation Loss,F1
1,0.617159,0.518529,0.417690
2,0.446055,0.510449,0.469388
3,0.337714,0.579753,0.495726


Best hyperparameters for Deberta: (2e-05, 16), Average F1: 0.5295
{'Roberta': {'best_hps': (2e-05, 16), 'avg_f1': 0.543554147993319}, 'Deberta': {'best_hps': (2e-05, 16), 'avg_f1': 0.52946600830309}}


In [21]:
tuning_res = {
    'Roberta': {
        'best_hps': (2e-05, 16),
        'avg_f1': 0.543554147993319
    },
    'Deberta': {
        'best_hps': (2e-05, 16),
        'avg_f1': 0.52946600830309
    }
}


In [ ]:
FINAL_EPOCHS = 10

# Retrain best models on full training set
def train_ensemble_models(tuning_results):
    for name, checkpoint, tokenizer in architectures:
        print(f"Retraining {name} on full training set...")

        trds = Dataset.from_pandas(trdf)
        valds = Dataset.from_pandas(valdf)

        trds = trds.map(lambda batch: tokenize(tokenizer, batch), batched=True)
        valds = valds.map(lambda batch: tokenize(tokenizer, batch), batched=True)
        trds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
        valds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

        lr, bsz = tuning_results[name]["best_hps"]

        model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2,
                                                                   hidden_dropout_prob=DROPOUT,
                                                                   attention_probs_dropout_prob=DROPOUT).to(device)
        training_args = TrainingArguments(
            output_dir=f'./final_results/{name}',
            learning_rate=lr,
            per_device_train_batch_size=bsz,
            per_device_eval_batch_size=bsz,
            num_train_epochs=FINAL_EPOCHS,
            eval_strategy="epoch",
            logging_strategy="epoch",
            save_strategy="best",
            save_total_limit=1,
            metric_for_best_model="f1",
            weight_decay=WEIGHT_DECAY
        )

        trainer = BalancedTrainer(
            model=model,
            args=training_args,
            train_dataset=trds,
            eval_dataset=valds,
            compute_metrics=compute_metrics
        )

        trainer.train()
    

In [ ]:
train_ensemble_models(tuning_res)


Retraining Roberta on full training set...


Map:   0%|          | 0/8252 [00:00<?, ? examples/s]

Map:   0%|          | 0/838 [00:00<?, ? examples/s]

Loading weights:   0%|          | 0/197 [00:00<?, ?it/s]

RobertaForSequenceClassification LOAD REPORT from: FacebookAI/roberta-base
Key                             | Status     | 
--------------------------------+------------+-
lm_head.bias                    | UNEXPECTED | 
lm_head.layer_norm.weight       | UNEXPECTED | 
lm_head.dense.bias              | UNEXPECTED | 
lm_head.layer_norm.bias         | UNEXPECTED | 
roberta.embeddings.position_ids | UNEXPECTED | 
lm_head.dense.weight            | UNEXPECTED | 
classifier.out_proj.weight      | MISSING    | 
classifier.out_proj.bias        | MISSING    | 
classifier.dense.bias           | MISSING    | 
classifier.dense.weight         | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


...

model.safetensors:   0%|          | 0.00/559M [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss,F1
1,0.527391,0.569543,0.350000
2,0.368904,0.546793,0.496124
3,0.275895,0.837399,0.466667
4,0.204873,0.631580,0.554913
5,0.147570,1.099246,0.508475
6,0.086801,1.148908,0.492308
7,0.059643,1.482362,0.451613
8,0.043804,1.516478,0.452174
9,0.032582,1.588402,0.462810
10,0.020745,1.571118,0.453782


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
ROB_CHECKPOINT = None
DEB_CHECKPOINT = None

ROB_F1 = None
DEB_F1 = None

ensemble_models = {}

for name, _, tokenizer in architectures:
    if name == "Roberta":
        checkpoint = ROB_CHECKPOINT
        f1 = ROB_F1
    else:
        checkpoint = DEB_CHECKPOINT
        f1 = DEB_F1

    model = AutoModelForSequenceClassification.from_pretrained(checkpoint).to(device)
    ensemble_models[name] = (model, tokenizer, f1 / (ROB_F1 + DEB_F1))



Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/200 [00:00<?, ?it/s]

In [65]:
from torch.utils.data import DataLoader

BSZ=16

def ensemble_predict(dataset, ensemble_models):
    dataloader = DataLoader(dataset, batch_size=BSZ)
    all_preds = []

    for batch in dataloader:
        texts = batch["text"]
        batch_probs = None

        for model, tokenizer, weight in ensemble_models.values():
            # Tokenize this batch for this model
            encodings = tokenizer(
                texts,
                padding=True,
                truncation=True,
                return_tensors="pt"
            )
            input_ids = encodings["input_ids"].to(device)
            attention_mask = encodings["attention_mask"].to(device)

            model.to(device)
            model.eval()
            with torch.no_grad():
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                probs = F.softmax(outputs.logits, dim=-1) * weight

            # Accumulate weighted probabilities
            if batch_probs is None:
                batch_probs = probs
            else:
                batch_probs += probs

            # Move model back to CPU to free GPU memory
            model.to("cpu")

        # Final predictions for this batch
        preds = torch.argmax(batch_probs, dim=-1)
        all_preds.extend(preds.cpu().numpy())

    return all_preds

In [66]:
# Helper function to save predictions to an output file
def labels2file(p, outf_path):
    with open(outf_path, "w") as f:
        for pred in p:
            f.write(f"{pred}\n")

In [ ]:
import os

def dev_predict(models, path, df=devdf):
    devds = Dataset.from_pandas(df)

    # Generate dev set predictions using ensemble
    preds = ensemble_predict(devds, models)
    labels2file(preds, os.path.join('res/', path))

    # Save reference labels for dev set
    labels2file(df['labels'].tolist(), os.path.join('ref/', path))

    !python3 evaluation.py . . $path
    !cat scores.txt


In [ ]:
dev_predict(ensemble_models, 'dev.txt')

In [ ]:
dpm.load_test()

cols = ['par_id', 'text', 'label']
testdf = dpm.test_set_df.copy()
testdf['text'] = testdf['keyword'] + ' ' + testdf['text']
testdf = testdf[cols]
testdf.rename(columns={"label": "labels"}, inplace=True)
print(testdf.head())

testds = Dataset.from_pandas(testdf)

# Generate test set predictions
preds = ensemble_predict(testds, ensemble_models)
labels2file(preds, os.path.join('res/', 'test.txt'))

  par_id                                               text  labels
0    t_4  Members of the church , which is part of Ken C...       1
1    t_5  "To ensure that "" Priority Agriculture Progra...       1
2    t_6  The deportees stepped off their flight from El...       1
3    t_7  PIMS staffer who raped disabled girl at ICU wa...       1
4    t_9  I conclude , Yes , the general FEELING generat...       1


# Ablation Studies #

In [ ]:
FINAL_EPOCHS = 10
WEIGHT_DECAY = 0.05
DROPOUT = 0.2

def train_ablation(tuning_results, directory, trdf, valdf, TrainerClass=BalancedTrainer):
    name, checkpoint, tokenizer = architectures[0]

    trds = Dataset.from_pandas(trdf)
    valds = Dataset.from_pandas(valdf)

    trds = trds.map(lambda batch: tokenize(tokenizer, batch), batched=True)
    valds = valds.map(lambda batch: tokenize(tokenizer, batch), batched=True)
    trds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
    valds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

    lr, bsz = tuning_results[name]["best_hps"]

    model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2,
                                                               hidden_dropout_prob=DROPOUT,
                                                               attention_probs_dropout_prob=DROPOUT).to(device)
    training_args = TrainingArguments(
        output_dir=f'./ablation_results/{directory}',
        learning_rate=lr,
        per_device_train_batch_size=bsz,
        per_device_eval_batch_size=bsz,
        num_train_epochs=FINAL_EPOCHS,
        eval_strategy="epoch",
        logging_strategy="epoch",
        save_strategy="best",
        save_total_limit=1,
        metric_for_best_model="f1",
        weight_decay=WEIGHT_DECAY
    )

    trainer = TrainerClass(
        model=model,
        args=training_args,
        train_dataset=trds,
        eval_dataset=valds,
        compute_metrics=compute_metrics
    )

    trainer.train()


## Unweighted Cross Entropy Loss ##

In [ ]:
class UnbalancedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels").view(-1)

        outputs = model(**inputs)
        logits = outputs.logits.view(-1, 2)

        loss_fn = nn.CrossEntropyLoss()
        loss = loss_fn(logits, labels)

        return (loss, outputs) if return_outputs else loss

In [ ]:
directory = "unb"
train_ablation(tuning_res, directory, trdf, valdf, TrainerClass=UnbalancedTrainer)

In [ ]:
checkpoint = f"./ablation_results/{directory}/checkpoint-"
unbalanced_model = AutoModelForSequenceClassification.from_pretrained(checkpoint).to(device)

unb_models = {
    "Roberta": (unbalanced_model, rob_tokenizer, 1),
}

dev_predict(unb_models, 'dev-un.txt')

## Individual Models in Ensemble ##

In [ ]:
models = {}
models["Roberta"] = ensemble_models["Roberta"]

dev_predict(models, 'dev-rob.txt')

task1_precision:0.625
task1_recall:0.4430379746835443
task1_f1:0.5185185185185185


In [ ]:
models = {}
models["Deberta"] = ensemble_models["Deberta"]

dev_predict(models, 'dev-deb.txt')

## Without Synonym Augmentation ##

In [ ]:
no_aug_trdf = og_trdf.copy()
no_aug_devdf = og_devdf.copy()

# Keep keyword metadata
no_aug_trdf['text'] = no_aug_trdf['keyword'] + ' ' + no_aug_trdf['text']
no_aug_trdf.drop(columns=['keyword'], inplace=True)
no_aug_devdf['text'] = no_aug_devdf['keyword'] + ' ' + no_aug_devdf['text']
no_aug_devdf.drop(columns=['keyword'], inplace=True)

no_aug_trdf, no_aug_valdf = train_test_split(no_aug_trdf, test_size=VAL_SIZE, random_state=RANDOM_SEED, stratify=no_aug_trdf['labels'])

directory = "no_aug"

train_ablation(tuning_res, directory, no_aug_trdf, no_aug_valdf)

In [ ]:
checkpoint = f"./ablation_results/{directory}/checkpoint-"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2,
                                                           hidden_dropout_prob=DROPOUT,
                                                           attention_probs_dropout_prob=DROPOUT).to(device)

In [ ]:
models = {
    "Roberta": (model, rob_tokenizer, 1),
}

dev_predict(models, 'dev-no-aug.txt', df=no_aug_devdf)

## Without Keyword Metadata ##

In [ ]:
no_key_trdf = og_trdf.copy()
no_key_devdf = og_devdf.copy()

no_key_trdf, no_key_valdf = train_test_split(no_key_trdf, test_size=VAL_SIZE, random_state=RANDOM_SEED, stratify=no_key_trdf['labels'])

no_key_trdf = augment(no_key_trdf)

directory = "no_key"

train_ablation(tuning_res, directory, no_key_trdf, no_key_valdf)

In [ ]:
checkpoint = f"./ablation_results/{directory}/checkpoint-"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2,
                                                           hidden_dropout_prob=DROPOUT,
                                                           attention_probs_dropout_prob=DROPOUT).to(device)

In [ ]:
models = {
    "Roberta": (model, rob_tokenizer, 1),
}

dev_predict(models, 'dev-no-key.txt', df=no_key_devdf)